In [235]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import os
import pyodbc
import math
from datetime import datetime
from sqlalchemy import create_engine, exc
from dotenv import load_dotenv


load_dotenv()

True

In [236]:
investment = 'MARA'
risk_free_rate = 0.043

In [237]:
# Define the connection string for Windows Authentication
server = 'MoemenLaptop'
database = 'InvestmentPortfolio'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(connection_string)

# query
query = f"""SELECT [open], [close] 
            FROM monthly_data
            WHERE Symbol = '{investment}'""" 

# Use pandas to read the SQL query into a DataFrame
df = pd.read_sql(query, conn)

# Close the connection
conn.close()

df['return'] = df['close'] - df['open']

df

C:\Users\moeme\AppData\Local\Temp\ipykernel_11964\4171543688.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,open,close,return
0,3.58,7.21,3.63
1,7.17,7.10,-0.07
2,7.18,8.72,1.54
3,8.54,10.07,1.53
4,9.76,9.79,0.03
5,9.66,13.86,4.20
6,14.05,17.37,3.32
7,16.86,12.57,-4.29
8,12.53,8.50,-4.03
9,9.20,8.81,-0.39


In [238]:
position_open = df['open'].iloc[0]
position_close = df['close'].iloc[-1]
periods = df.shape[0]
periods

18

In [239]:
# portfolio return
Rx = ((position_close-position_open)/position_open)*100
Rf = risk_free_rate *100
stdv = df['return'].std()

sharpe_ratio = (Rx-Rf)/stdv

sharpe_ratio

np.float64(90.54764581657298)

# volatility


In [240]:
volatility = stdv * math.sqrt(periods)

volatility

np.float64(21.092834160946136)

In [241]:

df['diff'] = df['close'] - df['close'].mean()

df['diff'] = df['diff'] ** 2

sum = df['diff'].sum()

math.sqrt(sum/periods) * math.sqrt(periods)


24.759346607785194

# BETA

In [242]:
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(connection_string)

# query
query = f"""SELECT * 
            FROM monthly_data
            WHERE symbol = 'QQQ'""" 

# Use pandas to read the SQL query into a DataFrame
df_ = pd.read_sql(query, conn)

# Close the connection
conn.close()

df_['returns'] = df_['Open'] - df_['Close']

df_

C:\Users\moeme\AppData\Local\Temp\ipykernel_11964\3044001164.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ = pd.read_sql(query, conn)


,Symbol,Date,Open,Close,High,Low,returns
0,QQQ,2023-01-01,268.65,294.62,298.26,260.34,-25.97
1,QQQ,2023-02-01,294.41,293.56,313.68,290.05,0.85
2,QQQ,2023-03-01,293.26,320.93,321.17,285.19,-27.67
3,QQQ,2023-04-01,318.77,322.56,322.65,309.89,-3.79
4,QQQ,2023-05-01,322.09,347.99,353.93,315.12,-25.90
5,QQQ,2023-06-01,347.73,369.42,372.85,346.66,-21.69
6,QQQ,2023-07-01,370.07,383.68,387.98,363.41,-13.61
7,QQQ,2023-08-01,382.31,377.99,383.56,354.71,4.32
8,QQQ,2023-09-01,380.40,358.27,380.83,351.36,22.13
9,QQQ,2023-10-01,358.54,350.87,373.74,342.35,7.67


In [243]:
df = df.iloc[1:].reset_index(drop=True)

covariance = df['return'].cov(df_['returns'])
variance = df_['returns'].var()

beta = covariance/variance

beta

np.float64(-0.040345340024353575)